# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../WeatherPy/Output/Weather Data.csv")

df

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,Yellowknife,62.4560,-114.3525,41.56,78,75,17.27,CA,-21600
1,Dongsheng,39.8161,109.9776,47.57,51,62,9.98,CN,28800
2,Ürümqi,43.8010,87.6005,34.75,93,90,2.24,CN,28800
3,Albany,42.6001,-73.9662,40.03,88,87,3.69,US,-14400
4,Tiksi,71.6872,128.8694,1.20,97,100,2.33,RU,32400
...,...,...,...,...,...,...,...,...,...
544,Saint-Jérôme,45.7804,-74.0036,37.15,89,75,3.44,CA,-14400
545,Soyo,-6.1349,12.3689,80.65,67,70,8.66,AO,3600
546,Khorixas,-20.3667,14.9667,77.86,10,0,7.36,NaN,7200
547,Namtsy,62.7161,129.6658,30.31,98,97,4.97,RU,32400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
coordinates=[]

for index, row in df.iterrows():
    coordinates.append((row['Latitude'],row['Longitude']))

# coordinates

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

In [6]:
# Assign the marker layer to a variable
markers = gmaps.heatmap_layer(coordinates, weights=df['Humidity'],point_radius=15)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
tempmin = 60
tempmax = 80

df_temp_filt = df.loc[(df["Max Temperature"]<tempmax)&(df["Max Temperature"]<tempmin)]
df_temp_filt

hummax = 50
hummin = 30

df_hum_filt = df_temp_filt.loc[(df_temp_filt["Humidity"]<hummax)&(df_temp_filt["Humidity"]>hummin)]
df_hum_filt

windspeedmax = 10

df_windspeed_filt = df_hum_filt.loc[df_hum_filt['Wind Speed']<windspeedmax]
df_windspeed_filt

Cloudinessmax = 80

df_final = df_windspeed_filt.loc[df_windspeed_filt['Cloudiness']<Cloudinessmax]
df_final

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
209,Belyy Yar,53.6039,91.3903,42.69,49,0,6.71,RU,25200
298,Laas,46.6166,10.7002,54.01,31,12,0.60,IT,7200
375,Oga,46.4631,10.3470,46.63,33,3,1.21,IT,7200
430,Tongliao,43.6125,122.2653,48.99,35,0,6.71,CN,28800
443,Yinchuan,38.4681,106.2731,56.55,41,42,2.13,CN,28800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = df_final
# hotel_df["Hotel Name"]=""

# Dependencies
import requests
import json
import pprint
hotel_df

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
209,Belyy Yar,53.6039,91.3903,42.69,49,0,6.71,RU,25200
298,Laas,46.6166,10.7002,54.01,31,12,0.60,IT,7200
375,Oga,46.4631,10.3470,46.63,33,3,1.21,IT,7200
430,Tongliao,43.6125,122.2653,48.99,35,0,6.71,CN,28800
443,Yinchuan,38.4681,106.2731,56.55,41,42,2.13,CN,28800


In [9]:
hotel_list = []
hlat=[]
hlng=[]

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        # run a request using our params dictionary
        response = requests.get(base_url, params=params).json()
        hotelname = response["results"][0]["name"]
        CityName = row['City Name']
        print(hotelname)
        hotel_list.append(hotelname)
        hlat.append(response["results"][0]["geometry"]["location"]["lat"])
        hlng.append(response["results"][0]["geometry"]["location"]["lng"])

    except IndexError:
        print(f"No Hotel within 5km of this city")
        hotel_list.append("No Hotel within 5km of this city")
        hlat.append("")
        hlng.append("")

No Hotel within 5km of this city
Gasthof Hotel Sonne Silandro
QC Terme Bagni di Bormio Bagni Nuovi
Country Garden Phoenix Hotel
Kempinski Hotel Yinchuan


In [10]:
# Store Dataframe under a new name and reset index
hotel_reset = hotel_df
hotel_reset = hotel_reset.reset_index()
hotel_reset

,index,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,209,Belyy Yar,53.6039,91.3903,42.69,49,0,6.71,RU,25200
1,298,Laas,46.6166,10.7002,54.01,31,12,0.60,IT,7200
2,375,Oga,46.4631,10.3470,46.63,33,3,1.21,IT,7200
3,430,Tongliao,43.6125,122.2653,48.99,35,0,6.71,CN,28800
4,443,Yinchuan,38.4681,106.2731,56.55,41,42,2.13,CN,28800


In [11]:
# 
for x in range (0,len(hotel_reset['City Name'])):
    hotel_reset.loc[[x],["Hotel Name"]] = hotel_list[x]
    hotel_reset.loc[[x],["Hotel Lat"]] = hlat[x]
    hotel_reset.loc[[x],["Hotel Lng"]] = hlng[x]
hotel_reset=hotel_reset.rename(columns={'Latitude':'Lat','Longitude':'Lng',"City Name":"City"})
hotel_reset

,index,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name,Hotel Lat,Hotel Lng
0,209,Belyy Yar,53.6039,91.3903,42.69,49,0,6.71,RU,25200,No Hotel within 5km of this city,,
1,298,Laas,46.6166,10.7002,54.01,31,12,0.60,IT,7200,Gasthof Hotel Sonne Silandro,46.631183,10.75867
2,375,Oga,46.4631,10.3470,46.63,33,3,1.21,IT,7200,QC Terme Bagni di Bormio Bagni Nuovi,46.486378,10.360793
3,430,Tongliao,43.6125,122.2653,48.99,35,0,6.71,CN,28800,Country Garden Phoenix Hotel,43.653509,122.268609
4,443,Yinchuan,38.4681,106.2731,56.55,41,42,2.13,CN,28800,Kempinski Hotel Yinchuan,38.482094,106.22734


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_reset.iterrows()]
locations = hotel_reset[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…